In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!cp "/content/drive/MyDrive/Colab Notebooks/ntu_encoder_no_order.zip" "/content/"
#!ls -l "/content/"
import zipfile
zip_ref = zipfile.ZipFile("/content/ntu_encoder_no_order.zip", 'r')
zip_ref.extractall("/")
zip_ref.close()
!rm "/content/ntu_encoder_no_order.zip"

rm: cannot remove '/content/ntu_encoder.zip': No such file or directory


In [ ]:
import os
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot
import numpy as np
import sys

In [ ]:
def def_model():
  model = tf.keras.models.Sequential()
  model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=120,activation="softmax"))
  opt = tf.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig("/content/drive/MyDrive/Colab Notebooks/vgg16/" + filename + '_plot.png')
	pyplot.close()

In [ ]:
# run the test harness for evaluating a model
def evaluationModele():
  # définition du modèle
  model = def_model()
  ###
  train_batch_size=140
  test_batch_size=14
  train_datagen = ImageDataGenerator()
  test_datagen = ImageDataGenerator()
  # l'iterateur dans le dossier d'entrainement, génère des lots d'image augmentées et normalisées, le batch size est à 16 car on a assez peu d'images
  train_it = train_datagen.flow_from_directory('/content/ntu_encoder/train/',
    class_mode='categorical', batch_size=train_batch_size, target_size=(224, 224))
  # récupération des indices pour avoir une visualisation de comment est rangé le tout
  label_map = (train_it.class_indices)
  print(label_map)
  test_it = test_datagen.flow_from_directory('/content/ntu_encoder/test/',
    class_mode='categorical', batch_size=test_batch_size, target_size=(224, 224))
  # entrainement du modèle
  #checkpoint = ModelCheckpoint("/content/vgg16_1.h5", monitor="val_accuracy", verbose=0,save_best_only=True,save_weights_only=False,mode='auto',save_freq=1)
  #early = EarlyStopping(monitor='val_accuracy',min_delta=0,patience=20,verbose=0,mode='auto')
  hist=model.fit_generator(generator=train_it,steps_per_epoch=train_it.samples/train_it.batch_size,validation_data=test_it,validation_steps=test_it.samples/test_it.batch_size,epochs=5,verbose=1)
  # evaluation du modèle
  _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
  print('> %.3f' % (acc * 100.0))
  # learning curves
  summarize_diagnostics(hist)
  return model

In [ ]:
model = evaluationModele()
model.save("/content/drive/MyDrive/Colab Notebooks/vgg16/modelNtu120_entier.h5")

Found 4848172 images belonging to 120 classes.
{'apply cream on face': 0, 'apply cream on hand': 1, 'arm circles': 2, 'arm swings': 3, 'back pain': 4, 'blow nose': 5, 'bouce ball': 6, 'brush hair': 7, 'brush teeth': 8, 'bull up paper': 9, 'butt kicks': 10, 'capitulate': 11, 'carry object': 12, 'check time': 13, 'cheer up': 14, 'cheers and drink': 15, 'chest pain': 16, 'clapping': 17, 'couting money': 18, 'cross arms': 19, 'cross hands in front': 20, 'cross toe touch': 21, 'cutting nails': 22, 'cutting paper': 23, 'drink water': 24, 'drop': 25, 'eat meal': 26, 'exchange things': 27, 'falling down': 28, 'fan self': 29, 'flick hair': 30, 'fold paper': 31, 'follow': 32, 'giving object': 33, 'grab stuff': 34, 'hand waving': 35, 'headache': 36, 'high-five': 37, 'hit with object': 38, 'hopping': 39, 'hugging': 40, 'hush': 41, 'juggle bat tennis ball': 42, 'jump up': 43, 'kicking': 44, 'kicking something': 45, 'knock over': 46, 'make OK sign': 47, 'make victory sign': 48, 'move heavy objects':

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
34629/34629 [==============================] - 21267s 614ms/step - loss: 1.3418 - accuracy: 0.6244 - val_loss: 0.7403 - val_accuracy: 0.7688
Epoch 2/5
 6533/34629 [====>.........................] - ETA: 4:32:13 - loss: 0.6536 - accuracy: 0.7933